In [1]:
from keras.models import *
from keras.layers import *
from keras.callbacks import *
from keras.optimizers import *
from keras.applications import *
from keras.regularizers import *
from keras.preprocessing.image import *
from keras.utils.multi_gpu_utils import multi_gpu_model

from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot, plot_model

import time
from glob import glob
from tqdm import tqdm
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from sklearn.metrics import *

import multiprocessing
from multiprocessing.dummy import Pool
from pprint import pprint

Using TensorFlow backend.


In [2]:
os.environ['CUDA_VISIBLE_DEVICES'] = '0,1,2,3,4,5,6,7'
n_gpus = len(os.environ['CUDA_VISIBLE_DEVICES'].split(','))

epochs = [40, 20]
batch_size = 1 * n_gpus
preprocess_func = nasnet.preprocess_input
MODEL = NASNetLarge

f = 3.75
lr = 1e-3
l2_rate = 1e-5
width = int(2560 / f)
height = int(1920 / f)
classes = ['正常', '吊经', '擦洞', '跳花', '毛洞', '织稀', '扎洞', '缺经', '毛斑', '其他']
n_classes = len(classes)
now = time.strftime('%m%d')
model_name = f'{MODEL.__name__}_{now}_f{f}_l2{l2_rate}'
model_name

'NASNetLarge_0803_f3.75_l21e-05'

In [3]:
print('-'*80)
print('开始训练 %s 模型。' % model_name)
print(width, height, n_classes)
print(classes)
print('-'*80)

--------------------------------------------------------------------------------
开始训练 NASNetLarge_0803_f3.75_l21e-05 模型。
682 512 10
['正常', '吊经', '擦洞', '跳花', '毛洞', '织稀', '扎洞', '缺经', '毛斑', '其他']
--------------------------------------------------------------------------------


In [4]:
from keras import backend as K

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
K.set_session(tf.Session(config=config))

In [5]:
idg = ImageDataGenerator(horizontal_flip=True,
                         rotation_range=15,
                         zoom_range=0.2)
idg2 = ImageDataGenerator()

gen_train = idg.flow_from_directory('train', target_size=(height, width), 
                                    interpolation='lanczos', classes=classes, batch_size=batch_size)
gen_valid = idg2.flow_from_directory('valid', target_size=(height, width), shuffle=False, 
                                     interpolation='lanczos', classes=classes, batch_size=4)
y_valid = (gen_valid.classes != 0).astype(int)

Found 1617 images belonging to 10 classes.
Found 405 images belonging to 10 classes.


In [6]:
class RocAucMetricCallback(Callback):
    def __init__(self):
        super(RocAucMetricCallback, self).__init__()

    def on_epoch_end(self, epoch, logs={}):
        y_pred = model_parallel.predict_generator(gen_valid, verbose=1, workers=multiprocessing.cpu_count())
        y_pred2 = 1 - y_pred[:,0]
        y_pred3 = 1 - (np.argmax(y_pred, axis=-1) == 0)
        
        auc = roc_auc_score(y_valid, y_pred2)
        acc = accuracy_score(y_valid, y_pred3)
        logs['val_auc'] = auc
        logs['val_acc'] = acc
        logs['lr'] = float(K.get_value(model_parallel.optimizer.lr))
        print(f'ACC: {acc:.4f} AUC: {auc:.4f}')

In [7]:
base_model = MODEL(input_shape=(height, width, 3), include_top=False, weights='imagenet')

In [8]:
input_tensor = Input((height, width, 3))
x = Lambda(preprocess_func)(input_tensor)
x = base_model(x)
x = GlobalAvgPool2D()(x)
x = Dropout(0.5)(x)
x = Dense(n_classes, activation='softmax')(x)
model = Model(input_tensor, x)

In [9]:
for layer in base_model.layers + model.layers:
    if hasattr(layer, 'kernel_regularizer'):
        print(layer)
        layer.kernel_regularizer= l2(l2_rate)

In [10]:
model_parallel = multi_gpu_model(model, n_gpus)
model_parallel.__setattr__('callback_model', model)

In [11]:
plot_model(model, show_shapes=True, to_file=f'logs/plot_{model_name}.png')
# SVG(model_to_dot(model, show_shapes=True).create(prog='dot', format='svg'))

In [12]:
def train(start, end, lr, warmup=False):
    opt = SGD(lr, momentum=0.9, nesterov=True)

    callbacks = [
        RocAucMetricCallback(), 
        ModelCheckpoint('models/%s-{lr:.6f}-{epoch}-{val_auc:.6f}.h5' % model_name, 
                        save_weights_only=True), 
        CSVLogger('logs/%s.csv' % model_name, append=True), 
    ]
    
    model_parallel.compile(optimizer=opt, 
              loss='categorical_crossentropy',
              metrics=['accuracy'])

    model_parallel.fit_generator(gen_train, steps_per_epoch=len(gen_train),  
                                 workers=multiprocessing.cpu_count(), 
                                 initial_epoch=start, epochs=end, callbacks=callbacks)

In [ ]:
train(0, 10, 1e-4)

Epoch 1/10


In [13]:
start = 0
for i, epochs in enumerate(epochs):
    train(start, start + epochs, lr*0.1**i, warmup=i == 0)
    start += epochs

Epoch 1/40
102/102 [==============================] - 184s 2s/step
ACC: 0.6543 AUC: 0.5000
Epoch 2/40
102/102 [==============================] - 96s 943ms/step
ACC: 0.6543 AUC: 0.5000
Epoch 3/40
172/203 [========================>.....] - ETA: 1:06 - loss: 1.6666 - acc: 0.6533

KeyboardInterrupt: 

In [ ]:
y_pred = model.predict_generator(gen_valid, verbose=1, workers=multiprocessing.cpu_count())
y_pred2 = 1 - y_pred[:,0]
y_pred3 = 1 - (np.argmax(y_pred, axis=-1) == 0)
pprint(roc_auc_score(y_valid, y_pred2))
pprint(accuracy_score(y_valid, y_pred3))
pprint(confusion_matrix(y_valid > 0.5, y_pred3))